In [89]:
import pandas as pd
import numpy as np
from nba_api.stats.static import players
player_dict = players.get_players()

from nba_api.stats.static import teams 
teams = teams.get_teams()

from nba_api.stats.endpoints import leaguegamefinder
import time

In [90]:
#test dataframe for individual team
team_ids=[x['id'] for x in teams]
leaguegamefinder.LeagueGameFinder(team_id_nullable=team_ids[1]).get_data_frames()[0]

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [87]:
def import_data_all_teams(year_list, season_types):
    for team in team_ids:
        df = leaguegamefinder.LeagueGameFinder(team_id_nullable=team).get_data_frames()[0]
        # Conditions for playoffs, regular, and preseason
        conditions = [
            (df['SEASON_ID'].str.slice(stop=1) == '4'),
            (df['SEASON_ID'].str.slice(stop=1) == '2'),
            (df['SEASON_ID'].str.slice(stop=1) == '1')
            ]

        # list of the values to assign for each condition
        values = ['Playoffs', 'Regular', 'Preseason']

        # create a new column and use np.select to assign values to it using our lists as arguments
        df['SEASON_TYPE'] = np.select(conditions, values)

        # now that we have created a season type column, remove the unnecessary first digit from season_id
        df['SEASON_ID'] = df['SEASON_ID'].str.slice(start=1)

        # cast season_id as int to make it easy to work with
        df = df.astype({'SEASON_ID': 'int32'})
        
        # select the years we want
        df = df[df['SEASON_ID'].isin(year_list)]
        df = df[df['SEASON_TYPE'].isin(season_types)]

        # create dataframe with all teams
        if team == team_ids[0]:
            out = df
        else:
            out = out.append(df,ignore_index=True)
        time.sleep(0.3)
               
    return out

In [88]:
year_list = list(range(1996,2020))
#season_types = ['Regular','Playoffs']
season_types = ['Regular']
nba = import_data_all_teams(year_list, season_types)

ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)